Before you run this notebook, go to the "classification_interactive.ipynb" to train a model to use for the game. First make TASK = 'tic-tac-toe' and CATEGORIES = ['x', 'y', 'blank']. Then take pictures for each category following the notebook instructions, train the model, and save the model. I did 25 pictures for each category and 25 epochs of training for the demo in my YouTube video. But fell free to experiment to see how you can improve your accuracy.

Run this cell to view a live output from the webcam and have the tic-tac-toe board fill the camera frame while stationary. Then press stop to disable the live preview.

In [ ]:
#Code modified from https://stackoverflow.com/questions/34588464/python-how-to-capture-image-from-webcam-on-click-using-opencv
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
import threading

# Stop button
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)


# Display function
def view(button):
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    i = 0
    while True:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
        if stopButton.value==True:
            cap.release()
            del(cap)
            display_handle.update(None)

            
# Run
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

Run this cell to start the tic-tac-toe game and play against the computer.

In [ ]:
import torch
import torchvision
import cv2

import torchvision.transforms as transforms
import PIL.Image as Image

classes = ["x", "o", "blank"]

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, len(classes))
model.load_state_dict(torch.load('/nvdli-nano/data/classification/my_model.pth'))

image_transforms = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

name = ['1x1.jpg', '1x2.jpg', '1x3.jpg', '2x1.jpg', '2x2.jpg', '2x3.jpg', '3x1.jpg', '3x2.jpg', '3x3.jpg']

def classify(model, image_transforms, image_path, classes):
    model = model.eval()
    image = Image.open(image_path)
    image = image_transforms(image).float()
    image = image.unsqueeze(0)
    
    output = model(image)
    _, predicted = torch.max(output.data, 1)
    return(classes[predicted.item()])
    
def parse_image():
    # take image
    camera = cv2.VideoCapture(0)
    return_value, img = camera.read()
    #cv2.imwrite('tic-tac-toe.jpg', image)

    # read image
    #img = cv2.imread('tic-tac-toe.jpg', cv2.IMREAD_UNCHANGED)

    # get dimensions of image
    dimensions = img.shape

    # height, width, number of channels in image
    height = img.shape[0]
    width = img.shape[1]

    x1 = [0, round(width / 3), round(2 * (width / 3)), 0, round(width / 3), round(2 * (width / 3)), 0, round(width / 3), round(2 * (width / 3))]
    x2 = [round(width / 3), round(2 * (width / 3)), width, round(width / 3), round(2 * (width / 3)), width, round(width / 3), round(2 * (width / 3)), width]
    y1 = [0, 0, 0, round(height / 3), round(height / 3), round(height / 3), round(2 * (height / 3)), round(2 * (height / 3)), round(2 * (height / 3))]
    y2 = [round(height / 3), round(height / 3), round(height / 3), round(2 * (height / 3)), round(2 * (height / 3)), round(2 * (height / 3)), height, height, height]

    for i in range(9):
        crop_img = img[y1[i]:y2[i], x1[i]:x2[i]]
        cv2.imwrite(name[i], crop_img)
        
def play_game():
    # 0 = empty, 1 = X, 2 = O
    # 0, 1, 2
    # 3, 4, 5
    # 6, 7, 8
    board = [0, 0, 0, 0, 0, 0, 0, 0, 0]

    # Input variables for Tic-tac-toe game
    player = 1
    position = 0

    # Game loop
    while True:
        # Display board
        print(board[0], board[1], board[2])
        print(board[3], board[4], board[5])
        print(board[6], board[7], board[8])

        # Get player input
        if player == 1:
            input("Player " + str(player) + ", press enter to take image.")
            parse_image()
            
            for i in range(9):
                if classify(model, image_transforms, name[i], classes) == 'x' and board[i] == 0:
                    position = i
                    break
            else:
                print("No new move found with machine vision. Defaults to position zero if first move.")
                    
        else:
            # Tic-tac-toe AI
            if board[4] == 0:
                position = 4
            elif board[0] == 0:
                position = 0
            elif board[2] == 0:
                position = 2
            elif board[6] == 0:
                position = 6
            elif board[8] == 0:
                position = 8    
            elif board[1] == 0:
                position = 1
            elif board[3] == 0:
                position = 3
            elif board[5] == 0:
                position = 5
            elif board[7] == 0:
                position = 7

            print("Player " + str(player) + ", choose a position: " + str(position))

        # Check if position is empty
        if board[position] == 0:
            # Set position to player
            board[position] = player

            # Check if player won
            if board[0] == player and board[1] == player and board[2] == player:
                print("Player " + str(player) + " wins!")
                break
            elif board[3] == player and board[4] == player and board[5] == player:
                print("Player " + str(player) + " wins!")
                break
            elif board[6] == player and board[7] == player and board[8] == player:
                print("Player " + str(player) + " wins!")
                break
            elif board[0] == player and board[3] == player and board[6] == player:
                print("Player " + str(player) + " wins!")
                break
            elif board[1] == player and board[4] == player and board[7] == player:
                print("Player " + str(player) + " wins!")
                break
            elif board[2] == player and board[5] == player and board[8] == player:
                print("Player " + str(player) + " wins!")
                break
            elif board[0] == player and board[4] == player and board[8] == player:
                print("Player " + str(player) + " wins!")
                break
            elif board[2] == player and board[4] == player and board[6] == player:
                print("Player " + str(player) + " wins!")
                break

            # Check if board is full
            if 0 not in board:
                print("Draw!")
                break

            # Switch player
            if player == 1:
                player = 2
            else:
                player = 1
        else:
            print("Position already taken!")

play_game()